In [42]:

# imports
%matplotlib notebook
import sys
from astropy.table import Table
from astropy.io import fits
from linetools.spectra.xspectrum1d import XSpectrum1D


from specdb.specdb import UVQS
from astropy.table import Table
from astropy.io import ascii
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
import glob
import numpy.ma as ma
import math
from scipy.optimize import curve_fit
import pylab as plb
from scipy.optimize import fmin
from scipy import asarray as ar,exp
import PyAstronomy.pyasl as pyasl



In [43]:
def airtovac(wavelength):
    """ Convert air-based wavelengths to vacuum

    Parameters:
    ----------
    wave: ndarray
      Wavelengths in Ang

    Returns:
    ----------
    wave: Quantity array
      Wavelength array corrected to vacuum wavelengths
    """
    # Standard conversion format
    sigma_sq = (1.e4/wavelength)**2. #wavenumber squared
    factor = 1 + (5.792105e-2/(238.0185-sigma_sq)) + (1.67918e-3/(57.362-sigma_sq))
    factor = factor*(wavelength>=2000.) + 1.*(wavelength<2000.) #only modify above 2000A

    # Convert
    new_wave = wavelength*factor
    return new_wave

def vactoair(wavelength):
    # Standard conversion format
    sigma_sq = (1.e4/wavelength)**2. #wavenumber squared
    factor = 1 + (5.792105e-2/(238.0185-sigma_sq)) + (1.67918e-3/(57.362-sigma_sq))
    factor = factor*(wavelength>=2000.) + 1.*(wavelength<2000.) #only modify above 2000A

    # Convert
    new_wave = wavelength*factor
    return new_wave



## Convert CPD-56_F.fits into ascii table

In [46]:

sp = XSpectrum1D.from_file('/Users/catherinemanea/Documents/j0608/data/spec/CPD-56_F.fits') 
sp.write_to_ascii('CPD56.ascii')


/Users/catherinemanea/Documents/j0608/data/spec/linetools/linetools/spectra/xspectrum1d.py:108: UserWarning: Assuming wavelength unit is Angstroms
  warnings.warn("Assuming wavelength unit is Angstroms")


# ALIS reader

## Read and distribute all ALIS .mod.out info into lists

In [835]:
velocitycorrection = 247.3
c = 3*(10**8)
chars = "wave= =IntFluxredshiftoibamplitude"

files = glob.glob('/Users/catherinemanea/Documents/j0608/fits/*.mod.out')

#initiate lists (separate ones for absorption)
amplist = []
amperrlist = []
redlist = []
rederrlist = []
displist = []
disperrlist = []
restwavelist = []


for file in files:
    modoutfile = open(file,"r")
    lines = modoutfile.readlines()
    for line in lines:
        if "gaussian  " in line:
            if line[0] != "#":
                amplitude = line[13:29].strip(chars)
                amplist.append(float(amplitude))
                redshift = line[29:44].strip(chars)
                redlist.append(redshift)
                disp = line[44:60].strip(chars)
                if disp[1] == " " or disp[2] == " ":
                    disp = disp[2:].strip(chars)
                displist.append(float(disp))
                restwave = line[75:96].strip(chars)
                restwavelist.append(float(restwave))
                temp.append(float(restwave))
            if line[0] == "#" and line[2] != " ":
                amperr = line[14:29].strip(chars)
                amperrlist.append(float(amperr))
                rederr = line[30:45].strip(chars)
                rederrlist.append(rederr)
                disperr = line[45:62].strip(chars)
                if ' ' in disperr:
                    if 'E' in disperr:
                        disperr = disperr[6:].strip(chars)
                    else:
                        disperr = disperr[2:].strip(chars)
                disperrlist.append(float(disperr))
    modoutfile.close()
    
    
    
cleanredlist = []
cleanrederrlist = []


#convert redshifts into decimal form
for item in redlist:
    if 'E' in item:
        item = float(item[:-4])*(10**(-4))
    cleanredlist.append(float(item))
for item in rederrlist:
    if 'E' in item:
        if item[-1:] == "-" or item[-1:] == "E" or item[-1:]==0:
            item = float(item[:-4])*(10**(-6))
        else:
            item = float(item)
    cleanrederrlist.append(float(item))

tt = Table([restwavelist,displist,cleanredlist], 
                     names = ('Wave','Disp','Red'))

## Extract P Cygni Info

In [836]:

PCyg = []
PCygerr = []
PCygdisp = []
PCygdisperr = []
PCygred = []
PCygrederr = []
flagged = []
numms = []



for ii in range(125):
    if restwavelist[ii] == restwavelist[ii+1]:
        PCyg.append(np.round(float(amplist[ii]),4))
        PCygerr.append(np.round(float(amperrlist[ii]),4))
        PCygdisp.append(np.round(float(displist[ii]),4))
        PCygdisperr.append(np.round(float(disperrlist[ii]),4))
        PCygred.append(np.round(float(cleanredlist[ii]),4))
        PCygrederr.append(np.round(float(cleanrederrlist[ii]),4))
        numms.append(ii)

        restwavelist.remove(restwavelist[ii])
        amplist.remove(amplist[ii])
        amperrlist.remove(amperrlist[ii])
        displist.remove(displist[ii])
        disperrlist.remove(disperrlist[ii])
        cleanredlist.remove(cleanredlist[ii])
        cleanrederrlist.remove(cleanrederrlist[ii])
        flagged.append(ii)
        ii = ii+1
    else:
        PCyg.append(0)
        PCygerr.append(0)
        PCygdisp.append(0)
        PCygdisperr.append(0)
        PCygred.append(0)
        PCygrederr.append(0)
        ii = ii+1
PCyg.append(0)
PCygerr.append(0)
PCygdisp.append(0)
PCygdisperr.append(0)
PCygred.append(0)
PCygrederr.append(0)

#print(numms)
#newdisp = np.delete(np.array(displist), numms)

## Create linelist dict:

In [837]:
IDdict = {3727.1:"OII",   3730.1:"OII",   3889.75:"HeI",   3889.9:"HeI",   3920.1:"CII",   3921.8:"CII",   4102.85:"H-Delta",   4268.2:"CII",   4341.7:"H-Gamma",
 4369.3:"CII",   4371.9:"CII",   4373.6:"CII",   4375.5:"CII",   4376.2:"CII",   4377.8:"CII",   4410.4:"CII",   4411.2:"CII",
 4412.4:"CII",   4412.7:"CII",   4414.5:"CII",   4416.14:"CII",  4418.215:"CII", 4472.74:"HeI",   4619.69:"CII",  4620.49:"CII",
 4626.89:"CII",  4628.69:"CII",  4631.29:"CII",  4714.5:"HeI",   4728.7:"CII",   4735.9:"CII",   4736.8:"CII",   4739.3:"CII",
 4746.1:"CII",  4748.6:"CII",   4862.7:"H-Beta",   4868.5:"CII", 4923.3:"HeI",   5017.0:"HeI",   5032.1:"??",   5035.9:"CII",
 5040.7:"??",   5041.8:"CII",   5044.3:"CII",   5045.:"CII",    5047.1:"CII",   5049.2:"??",   5109.3:"CII",   5115.1:"CII",
 5115.7:"CII",   5118.1:"CII",   5120.8:"CII",   5121.5:"CII",   5123.2:"CII",   5123.9:"CII",   5126.6:"CII",   5128.3:"CII",
 5134.3:"CII",   5134.7:"CII",   5138.7:"CII",   5140.6:"CII",   5144.9:"CII",   5146.6:"CII",   5152.5:"CII",   5250.989:"CII",5251.45: "CII",
 5255.02:"CII",   5257.55:"CII",   5258.7:"CII",   5260.5:"CII",   5261.22:"CII",   5334.4:"CII",   5336.3:"CII",   5342.96:"CII",
 5369.94:"CII",  5370.03:"CII",  5374.355:"CII", 5536.5:"CII",   5539.5:"CII",   5642.1:"CII",   5649.7:"CII",   5664.1:"CII",
 5697.5:"CIII",   5877.25:"HeI",  6463.79:"CII",   6564.614:"H-Alpha", 6579.9:"CII",  6584.69:"NII",   6679.9:"HeI",
 6782.5:"CII",   6785.8:"CII",   6789.:"CII",    6793.4:"CII",   6800.:"CII",    6802.6:"CII",   6814.2:"CII",   7067.1:"HeI",
 7114.5:"CII",   7115.:"CII",    7117.6:"CII",   7127.7:"CII",   7134.4:"CII",   7136.1:"CII",   7146.2:"CII",   7233.3:"CII",
 7238.4:"CII",   7239.2:"CII",   7283.3:"HeI",   7321.:"OII",    7332.:"OII",    7507.73:"CII",   7510.96:"CII",    7521.6:"CII",
 7522.:"CII",    7532.64:"CII",   7771.5:"OI",   7777.5:"OI",   8031.1:"CII",   8039.9:"CII",   8041.6:"CII",   8050.5:"CII",
 8064.3:"CII",   8065.:"CII",    8078.8:"CII",   8448.8:"OI", 6579.87:"CII", 5369.83:"CII", 7773.9:"CIII?", 7774.08:"OI", 7776.31:"OI", 7777.53:"OI",
         5016.73: "HeI", 6152.45:"CII", 5017.08:"HeI", 5891.41: "CII", 5893.22: "CII", 6096.98: "CII", 6100.2:"CII", 6104.25:"CII", 6462.08: "CII"}



## Create ID column

In [838]:
IDs = []
for item in restwavelist:
    IDs.append(IDdict[item])

## Create table

In [839]:

amplist,redlist,displist,restwavelist = np.array(amplist),np.array(redlist),np.array(displist),np.array(restwavelist)
cleanredlist = np.array(cleanredlist)
PCyg , PCygdisp = np.array(PCyg), np.array(PCygdisp)

observedwavelist = restwavelist*(cleanredlist+1)

parsetable = Table([np.round(amplist,2),cleanredlist,np.round(displist,2),restwavelist,PCyg, PCygerr, PCygdisperr,np.round(amperrlist,4),cleanrederrlist,IDs,PCygred,np.round(PCygdisp,1)], 
                     names = ('ALIS Amplitude','Redshift','Dispersion','Rest Wavelength',"P Cygni Amp","P Cyg Err","P Cyg Disp Err",
                             "Amp Err","Redshift Err","ID","PCygRed","PCygDisp"))


## Create observed wavelength column

In [840]:
observedwaves = np.array(restwavelist*(cleanredlist+1))



parsetable["Observed Wavelength"] = np.round(observedwaves,1)
parsetable["Observed Wave Err"] = np.round(restwavelist*(cleanredlist+cleanrederrlist+1) - observedwaves,4)

## Velocity Column (based on doppler shift)

In [841]:
parsetable["Velocity"] = np.round(parsetable["Redshift"]*c/(10**3),0).astype(int)
parsetable["PCygV"] = np.round(parsetable["PCygRed"]*c/(10**3),0).astype(int)

## Create columns with values plus/minus error (unnecessary for now)

In [842]:
ii = 0
obserwav = []
pc = []
amp = []
dis = []
airobserved = pyasl.vactoair2(observedwaves)

for item in airobserved:
    obserwav.append(str(np.round(item,1))+"$\pm$"+str(parsetable["Observed Wave Err"][ii]))
    ii = ii+1

jj = 0
for item in PCyg:
    pc.append(str(item)+"$\pm$"+str(PCygerr[jj]))
    jj = jj+1
    
bb = 0
for item in amplist:
    amp.append(str(np.round(item,2))+"$\pm$"+str(np.round(amperrlist[bb],2)))
    bb = bb+1
    
cc = 0
for item in displist:
    dis.append(str(np.round(item,1))+"$\pm$"+str(np.round(disperrlist[cc],1)))
    cc = cc+1

parsetable["ObsWav"] = obserwav
parsetable["PC"] = pc
parsetable["amp"] = amp      
parsetable["dis"] = dis


## Create columns for amplitudes as ratio to H-Beta

In [843]:
parsetable["ratioamp"] = np.round(amplist/50.40253105,2)
ratiopc = []
for item in PCyg:
    ratiopc.append(np.round(float(item)/50.40253105,2))
parsetable["ratiopc"] = ratiopc

## Convert back to air for Dr. Margon

In [845]:


restwaveair = np.round(pyasl.vactoair2(restwavelist),1)
parsetable["Rest Wavelength (air)"] = restwaveair
zair = (observedwaves/restwaveair) - 1
#parsetable["Vair"] = np.round(zair*c/(10**3),0).astype(int)
parsetable["Observed Wavelength (air)"] = airobserved

## Convert to Latex Table

In [846]:
ascii.write(parsetable.group_by("Rest Wavelength")["ID","Rest Wavelength (air)","Velocity","ObsWav","ratioamp","dis","ratiopc","PCygV","PCygDisp"], Writer=ascii.Latex, latexdict=ascii.latex.latexdicts['template'])






\begin{tabletype}[tablealign]
preamble
\caption{caption}
\begin{tabular}{col_align}
header_start
ID & Rest Wavelength (air) & Velocity & ObsWav & ratioamp & dis & ratiopc & PCygV & PCygDisp \\
 &  &  &  &  &  &  &  &  \\
header_end
data_start
OII & 3726.0 & 217 & 3728.7$\pm$0.0036 & 0.85 & 28.3$\pm$1.4 & 0.0 & 0 & 0.0 \\
OII & 3729.0 & 203 & 3731.6$\pm$0.0021 & 1.54 & 47.1$\pm$1.0 & 0.0 & 0 & 0.0 \\
HeI & 3888.6 & 252 & 3891.9$\pm$0.0192 & 2.45 & 25.2$\pm$2.1 & -1.17 & 180 & 46.8 \\
CII & 3919.0 & 245 & 3922.2$\pm$0.0004 & 1.13 & 6.1$\pm$1.8 & 0.0 & 0 & 0.0 \\
CII & 3920.7 & 253 & 3924.0$\pm$0.0003 & 1.69 & 21.2$\pm$0.6 & 0.0 & 0 & 0.0 \\
H-Delta & 4101.7 & 251 & 4105.1$\pm$0.0026 & 0.22 & 31.2$\pm$3.2 & 0.0 & 0 & 0.0 \\
CII & 4267.0 & 268 & 4270.8$\pm$0.0001 & 3.82 & 33.7$\pm$0.9 & 0.0 & 0 & 0.0 \\
H-Gamma & 4340.5 & 236 & 4343.9$\pm$0.0039 & 0.26 & 14.3$\pm$3.4 & 0.0 & 0 & 0.0 \\
CII & 4368.1 & 243 & 4371.6$\pm$0.0297 & 0.36 & 131.3$\pm$17.1 & -0.39 & 120 & 36.3 \\
CII & 4370.7 & 280